# Import libraries and set seeds

In [37]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.optim import optimize_acqf
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import LogExpectedImprovement
from botorch.utils.transforms import standardize
import numpy as np

def set_seeds(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seeds(42)

# Load the data

In [45]:
set_seeds(42)

# read the data from train_XY.csv and convert to a tensor
train_XY = np.loadtxt('train_XY.csv', delimiter=',')
train_X = torch.tensor(train_XY[:, :-1], dtype=torch.float)
train_Y = torch.tensor(train_XY[:, -1], dtype=torch.float)

print(train_X.shape)
print(train_Y.shape)

print(f"type of train_X: {type(train_X)}")
print(f"type of train_Y: {type(train_Y)}")

print(train_X)
print(train_Y)

torch.Size([4, 4])
torch.Size([4])
type of train_X: <class 'torch.Tensor'>
type of train_Y: <class 'torch.Tensor'>
tensor([[-1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00],
        [-2.7756e-16,  0.0000e+00,  5.5511e-16,  0.0000e+00],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 9.9806e-01,  7.6469e-01,  9.5246e-01,  8.4701e-01]])
tensor([-1.0000,  0.0000,  1.0000,  0.9900])


# Train the model

In [39]:
set_seeds(42)

# Standardize X and Y
train_X = standardize(train_X)
train_Y = standardize(train_Y)

# Define the GP model
gp = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)

# Fit the model
fit_gpytorch_mll(mll)

# define the acquisition function use Expected Improvement
EI = LogExpectedImprovement(gp, best_f=train_Y.max())

# Define the bounds for the variables
bounds = torch.tensor([
    [0.0, 0.0, 0.0, 0.0],  # Lower bounds for X1, X2, X3, X4
    [1.0, 1.0, 1.0, 1.0],  # Upper bounds for X1, X2, X3, X4
])

# Optimize the acquisition function
candidate, acq_value = optimize_acqf(
    acq_function=EI,
    bounds=bounds,
    q=1,  # Number of candidates to sample
    num_restarts=5,
    raw_samples=20,
)

# Print the next point to sample
print("Next point to sample: ", candidate)

Next point to sample:  tensor([[0.9981, 0.7647, 0.9525, 0.8470]])


/opt/miniconda3/envs/chemcar/lib/python3.11/site-packages/botorch/models/utils/assorted.py:257: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(


# Add new points to the dataset

In [40]:
# add the points from the candidate to the training data as double 
train_X = torch.cat([train_X, candidate], dim=0)

Y = 0.99

train_Y = torch.cat([train_Y, torch.tensor([[Y]], dtype=torch.float64)], dim=0)

Shape of train_X: torch.Size([4, 4])
type of train_X: torch.float64
Shape of train_Y: torch.Size([4, 1])
type of train_Y: torch.float64


# Save the updated data to a csv file

In [42]:
# combine the train_X and train_Y to a single tensor
train_XY = torch.cat([train_X, train_Y], dim=1)

# save the train_XY to a csv file 
np.savetxt("train_XY.csv", train_XY.numpy(), delimiter=",")